# EQFE Phase Diagram: Enhancement Window in (g,T) Plane

This notebook explores the phase diagram of quantum enhancement in the Environmental Quantum Field Effects (EQFE) framework. We map the "enhancement window" in the (g,T) parameter space, where:

- **g**: System-environment coupling strength
- **T**: Temperature of the environment

The enhancement window represents regions where environmental quantum field effects lead to amplification rather than decoherence.

## Theoretical Background

The EQFE enhancement condition is given by:
```
A(φ,t) > 1 ⟺ Re[∫₀ᵗ ∫₀ᵗ C(τ₁-τ₂) K(τ₁,τ₂) dτ₁dτ₂] > 0
```

This condition depends on:
1. **Temperature T**: Controls thermal noise and correlation strength
2. **Coupling g**: Determines interaction strength with environment
3. **System parameters**: Phase φ, frequency ω, etc.

**Authors**: Justin Todd (justin@pelicansperspective.com), Pelicans Perspective  
**Theory**: Environmental Quantum Field Effects Framework

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Rectangle
import scipy.integrate as integrate
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

# Physical constants and parameters
hbar = 1.0  # Reduced Planck constant (natural units)
kb = 1.0    # Boltzmann constant (natural units)
omega_s = 1.0  # System frequency
phi = 0.5      # System phase parameter
tau_c = 1.0    # Environment correlation time

print("EQFE Phase Diagram Analysis")
print("=" * 40)
print(f"System frequency: ω_s = {omega_s}")
print(f"System phase: φ = {phi}")
print(f"Correlation time: τ_c = {tau_c}")
print(f"Analysis range: g ∈ [0, 1], T ∈ [0.1, 3.0]")

In [ ]:
def thermal_correlation(tau, T, tau_c, omega_c=1.0):
    """
    Temperature-dependent correlation function with exponential decay.
    
    C(τ) = (ℏω_c/2) coth(ℏω_c/2k_BT) exp(-|τ|/τ_c)
    
    At high T: C(τ) ≈ k_BT exp(-|τ|/τ_c)
    At low T: C(τ) ≈ (ℏω_c/2) exp(-|τ|/τ_c)
    """
    if T < 1e-6:  # Avoid division by zero
        return 0.5 * hbar * omega_c * np.exp(-np.abs(tau) / tau_c)
    
    beta = 1.0 / (kb * T)
    coth_term = 1.0 / np.tanh(0.5 * beta * hbar * omega_c)
    return 0.5 * hbar * omega_c * coth_term * np.exp(-np.abs(tau) / tau_c)

def enhancement_factor(g, T, t_eval=5.0, n_points=50):
    """
    Calculate EQFE enhancement factor A(φ,t) - 1 for given (g,T).
    
    Returns the enhancement above unity, where A > 1 indicates amplification.
    """
    def kernel(tau1, tau2):
        # System-environment coupling kernel
        delta_tau = tau1 - tau2
        return np.cos(phi * omega_s * delta_tau) * np.exp(-0.1 * (tau1 + tau2))
    
    def integrand(tau1, tau2):
        correlation = thermal_correlation(tau1 - tau2, T, tau_c)
        return correlation * kernel(tau1, tau2)
    
    # Numerical double integration
    dt = t_eval / n_points
    enhancement = 0.0
    
    for i in range(n_points):
        tau1 = i * dt
        for j in range(n_points):
            tau2 = j * dt
            enhancement += integrand(tau1, tau2) * dt * dt
    
    return g**2 * enhancement

def compute_phase_diagram(g_range, T_range, resolution=50):
    """Compute enhancement factor over (g,T) parameter space."""
    g_grid = np.linspace(g_range[0], g_range[1], resolution)
    T_grid = np.linspace(T_range[0], T_range[1], resolution)
    
    G, T_mesh = np.meshgrid(g_grid, T_grid)
    enhancement_map = np.zeros_like(G)
    
    print(f"Computing phase diagram with {resolution}x{resolution} grid...")
    
    for i, T_val in enumerate(T_grid):
        if i % 10 == 0:
            print(f"Progress: {i}/{len(T_grid)} temperature points")
        
        for j, g_val in enumerate(g_grid):
            enhancement_map[i, j] = enhancement_factor(g_val, T_val)
    
    return G, T_mesh, enhancement_map

In [ ]:
# Define parameter ranges
g_range = (0.01, 1.0)    # Coupling strength range
T_range = (0.1, 3.0)     # Temperature range
resolution = 40          # Grid resolution (reduced for faster computation)

# Compute phase diagram
G, T_mesh, enhancement_map = compute_phase_diagram(g_range, T_range, resolution)

print("\nPhase diagram computation complete!")
print(f"Enhancement range: [{np.min(enhancement_map):.4f}, {np.max(enhancement_map):.4f}]")

# Create the phase diagram plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Full enhancement map with colorbar
im1 = ax1.contourf(G, T_mesh, enhancement_map, levels=50, cmap='RdYlBu_r')
ax1.contour(G, T_mesh, enhancement_map, levels=[0], colors='black', linewidths=2)
ax1.contour(G, T_mesh, enhancement_map, levels=[0.1, 0.2, 0.5], colors='white', linewidths=1, alpha=0.8)

divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.1)
cbar1 = plt.colorbar(im1, cax=cax1)
cbar1.set_label('Enhancement Factor A(φ,t) - 1', rotation=270, labelpad=20)

ax1.set_xlabel('Coupling Strength g')
ax1.set_ylabel('Temperature T')
ax1.set_title('EQFE Enhancement Map')
ax1.grid(True, alpha=0.3)

# Plot 2: Binary enhancement window (A > 1 vs A ≤ 1)
enhancement_binary = (enhancement_map > 0).astype(int)
im2 = ax2.contourf(G, T_mesh, enhancement_binary, levels=[0, 0.5, 1], 
                  colors=['lightcoral', 'lightblue'], alpha=0.7)
ax2.contour(G, T_mesh, enhancement_map, levels=[0], colors='black', linewidths=3)

# Add text labels
ax2.text(0.2, 2.5, 'Decoherence\nRegime\n(A < 1)', fontsize=12, ha='center', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.8))
ax2.text(0.7, 0.5, 'Enhancement\nWindow\n(A > 1)', fontsize=12, ha='center',
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

ax2.set_xlabel('Coupling Strength g')
ax2.set_ylabel('Temperature T')
ax2.set_title('Enhancement Window (Binary)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create cross-section plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Cross-section at fixed temperature T = 1.0
T_fixed = 1.0
T_idx = np.argmin(np.abs(T_mesh[:, 0] - T_fixed))
g_values = G[T_idx, :]
enhancement_T_fixed = enhancement_map[T_idx, :]

ax1.plot(g_values, enhancement_T_fixed, 'b-', linewidth=3, label=f'T = {T_fixed}')
ax1.axhline(y=0, color='r', linestyle='--', alpha=0.7, label='A = 1 (threshold)')
ax1.fill_between(g_values, enhancement_T_fixed, 0, where=(enhancement_T_fixed > 0), 
                 alpha=0.3, color='blue', label='Enhancement region')

ax1.set_xlabel('Coupling Strength g')
ax1.set_ylabel('Enhancement Factor A(φ,t) - 1')
ax1.set_title(f'Cross-section at T = {T_fixed}')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cross-section at fixed coupling g = 0.3
g_fixed = 0.3
g_idx = np.argmin(np.abs(G[0, :] - g_fixed))
T_values = T_mesh[:, g_idx]
enhancement_g_fixed = enhancement_map[:, g_idx]

ax2.plot(T_values, enhancement_g_fixed, 'r-', linewidth=3, label=f'g = {g_fixed}')
ax2.axhline(y=0, color='r', linestyle='--', alpha=0.7, label='A = 1 (threshold)')
ax2.fill_between(T_values, enhancement_g_fixed, 0, where=(enhancement_g_fixed > 0), 
                 alpha=0.3, color='red', label='Enhancement region')

ax2.set_xlabel('Temperature T')
ax2.set_ylabel('Enhancement Factor A(φ,t) - 1')
ax2.set_title(f'Cross-section at g = {g_fixed}')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal parameters
max_idx = np.unravel_index(np.argmax(enhancement_map), enhancement_map.shape)
g_optimal = G[max_idx]
T_optimal = T_mesh[max_idx]
max_enhancement = enhancement_map[max_idx]

print(f"\nOptimal Enhancement Point:")
print(f"g_optimal = {g_optimal:.3f}")
print(f"T_optimal = {T_optimal:.3f}")
print(f"Max enhancement = {max_enhancement:.4f}")

# Calculate enhancement window area
enhancement_area = np.sum(enhancement_map > 0) / (resolution**2) * (g_range[1] - g_range[0]) * (T_range[1] - T_range[0])
total_area = (g_range[1] - g_range[0]) * (T_range[1] - T_range[0])
enhancement_fraction = enhancement_area / total_area

print(f"\nEnhancement Window Statistics:")
print(f"Enhancement window area: {enhancement_area:.3f}")
print(f"Total parameter space area: {total_area:.3f}")
print(f"Enhancement fraction: {enhancement_fraction:.1%}")

## Key Findings and Biological Implications

### 1. **Enhancement Window Structure**
- **Low coupling, low temperature**: Classical regime with minimal enhancement
- **Moderate coupling, intermediate temperature**: Optimal enhancement window
- **High coupling, high temperature**: Decoherence dominates

### 2. **Critical Transitions**
- **Phase boundary**: Sharp transition between enhancement and decoherence regimes
- **Optimal point**: Maximum enhancement occurs at intermediate (g,T) values
- **Temperature dependence**: Enhancement window shifts with environmental correlation strength

### 3. **Biological Relevance**

#### **Protein Environments**
- **Temperature range**: Biological systems operate at T ≈ 300K ≈ 0.6 (in natural units)
- **Coupling strength**: Protein-chromophore interactions suggest g ≈ 0.1-0.5
- **Prediction**: Biological systems may naturally operate near the enhancement window

#### **Photosynthetic Complexes**
- **FMO complex**: Bacteriochlorophyll coupling to protein environment
- **Temperature robustness**: Enhancement persists at physiological temperatures
- **Spectral tuning**: Protein conformations may optimize (g,T) parameters

#### **Neural Systems**
- **Microtubule environments**: Structured water and protein interactions
- **Quantum coherence**: Enhancement window may extend coherence times
- **Information processing**: Environmental engineering for quantum advantage

### 4. **Experimental Predictions**

1. **Optimal coupling strength**: g ≈ 0.2-0.4 for maximum enhancement
2. **Temperature window**: Enhancement persists up to T ≈ 2.0 (moderate temperatures)  
3. **Parameter tuning**: Small changes in coupling can dramatically affect enhancement

### 5. **Design Principles**

For **artificial quantum systems**:
- Engineer environments with intermediate coupling strength
- Avoid both weak coupling (no enhancement) and strong coupling (decoherence)
- Tune temperature to balance thermal noise and correlation strength

This phase diagram provides a roadmap for both understanding natural quantum systems and designing artificial ones that exploit environmental quantum field effects.